## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-18-18-53-52 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/08/18/us/tru...
1,2025-08-18-18-53-23 +0000,nypost,Tallest subway platform in NYC among 60 statio...,https://nypost.com/2025/08/18/us-news/tallest-...
2,2025-08-18-18-51-32 +0000,nyt,Live Updates: Trump Says He’s Hoping to Meet W...,https://www.nytimes.com/live/2025/08/18/us/tru...
3,2025-08-18-18-46-07 +0000,bbc,Texas Democrats return home after redistrictin...,https://www.bbc.com/news/articles/cdd3gj5ve1go...
4,2025-08-18-18-40-02 +0000,bbc,What to know as Air Canada strike continues,https://www.bbc.com/news/articles/cq680gm5q86o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2339/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
16,trump,46
21,zelensky,24
41,ukraine,16
96,house,15
29,canada,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
181,2025-08-18-04-01-16 +0000,nypost,Trump says Zelensky can end war with Russia ‘a...,https://nypost.com/2025/08/18/world-news/trump...,141
140,2025-08-18-10-27-12 +0000,nypost,Harrowing photo shows teen killed in Russian s...,https://nypost.com/2025/08/18/world-news/russi...,129
133,2025-08-18-11-37-54 +0000,bbc,"'No going into Nato by Ukraine,' says Trump as...",https://www.bbc.com/news/articles/cm21j1ve817o...,126
23,2025-08-18-18-02-57 +0000,nypost,"Trump, Zelensky strike friendly tone as they t...",https://nypost.com/2025/08/18/us-news/trump-ze...,113
130,2025-08-18-12-03-49 +0000,wapo,"What to expect as Zelensky, European leaders m...",https://www.washingtonpost.com/world/2025/08/1...,112


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
181,141,2025-08-18-04-01-16 +0000,nypost,Trump says Zelensky can end war with Russia ‘a...,https://nypost.com/2025/08/18/world-news/trump...
106,52,2025-08-18-14-15-38 +0000,nypost,Illegal migrant suspect in deadly Florida cras...,https://nypost.com/2025/08/18/us-news/illegal-...
140,49,2025-08-18-10-27-12 +0000,nypost,Harrowing photo shows teen killed in Russian s...,https://nypost.com/2025/08/18/world-news/russi...
169,46,2025-08-18-06-34-45 +0000,nyt,Air Canada Flight Attendants Say They Will Def...,https://www.nytimes.com/2025/08/17/world/canad...
125,34,2025-08-18-12-31-57 +0000,nypost,Texas ICE shooting suspects tied to secretive ...,https://nypost.com/2025/08/18/us-news/texas-ic...
212,30,2025-08-17-21-45-24 +0000,nypost,New Yorkers fighting against massive battery s...,https://nypost.com/2025/08/17/us-news/new-york...
58,29,2025-08-18-16-44-00 +0000,wsj,"President Trump continues his on-again, off-ag...",https://www.wsj.com/politics/policy/trump-elec...
90,29,2025-08-18-15-18-49 +0000,nypost,Marvel leaves Georgia to film superhero movies...,https://nypost.com/2025/08/18/business/marvel-...
83,29,2025-08-18-15-31-36 +0000,bbc,'No-one was expecting this': European leaders ...,https://www.bbc.com/news/articles/c9d0v8ljw3no...
78,29,2025-08-18-15-52-15 +0000,cbc,Palestinians leave Gaza City fearing Israeli o...,https://www.cbc.ca/news/world/gaza-city-palest...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
